# Functions

In [1]:
import pandas as pd
import requests
from typing import Dict, List
import re
import json

def get_github_folder_files(username: str, repo_name: str, folder_path: str,
                           file_extension: str = ".csv") -> List[str]:
    """
    GitHub API를 사용해서 특정 폴더의 파일 목록을 가져오는 함수

    Parameters:
    -----------
    username : str
        GitHub 사용자명
    repo_name : str
        레포지토리 이름
    folder_path : str
        폴더 경로 (예: "temp/v1_g/3rd_2002")
    file_extension : str
        필터링할 파일 확장자 (기본값: ".csv")

    Returns:
    --------
    List[str]
        해당 폴더의 CSV 파일명 목록
    """

    # GitHub API URL
    api_url = f"https://api.github.com/repos/{username}/{repo_name}/contents/{folder_path}"

    try:
        response = requests.get(api_url)
        response.raise_for_status()

        files_data = response.json()

        # CSV 파일만 필터링
        csv_files = []
        for file_info in files_data:
            if file_info['type'] == 'file' and file_info['name'].endswith(file_extension):
                csv_files.append(file_info['name'])

        return sorted(csv_files)

    except requests.exceptions.RequestException as e:
        print(f"❌ GitHub API 요청 실패: {e}")
        return []
    except json.JSONDecodeError as e:
        print(f"❌ JSON 파싱 실패: {e}")
        return []


def create_variable_name_from_filename(filename: str) -> str:
    """
    파일명에서 변수명을 생성하는 함수
    temp1_governor_지역명_숫자.csv → g숫자_지역명

    Parameters:
    -----------
    filename : str
        파일명 (예: "temp1_governor_busan_3.csv", "temp1_governor_seoul_4.csv")

    Returns:
    --------
    str
        변수명 (예: "g3_busan", "g4_seoul")
    """

    # temp1_governor_지역명_숫자.csv → g숫자_지역명
    match = re.search(r'temp1_governor_([^_]+)_(\d+)\.csv', filename)
    if match:
        region = match.group(1)
        number = match.group(2)
        return f"g{number}_{region}"

    # 매칭되지 않으면 기본적으로 파일명 정리해서 반환
    base_name = filename.replace('.csv', '').replace('temp1_', '')
    return base_name


def load_all_csvs_from_github_folder(username: str, repo_name: str, folder_path: str) -> Dict[str, pd.DataFrame]:
    """
    GitHub 폴더에서 모든 CSV 파일을 자동으로 읽어와서 DataFrame 딕셔너리로 반환

    Parameters:
    -----------
    username : str
        GitHub 사용자명
    repo_name : str
        레포지토리 이름 (예: "korean-elections")
    folder_path : str
        폴더 경로 (예: "temp/v1_g/3rd_2002")

    Returns:
    --------
    Dict[str, pd.DataFrame]
        변수명을 키로 하고 DataFrame을 값으로 하는 딕셔너리
    """

    # 1. GitHub API로 파일 목록 가져오기
    csv_files = get_github_folder_files(username, repo_name, folder_path)

    if not csv_files:
        print("❌ CSV 파일을 찾을 수 없습니다.")
        return {}

    # 2. 각 CSV 파일 읽어오기
    base_url = f"https://raw.githubusercontent.com/{username}/{repo_name}/refs/heads/main/{folder_path}/"
    dataframes = {}

    for filename in csv_files:
        try:
            # URL 생성
            url = base_url + filename

            # 변수명 생성
            var_name = create_variable_name_from_filename(filename)

            # CSV 읽기
            df = pd.read_csv(url)

            dataframes[var_name] = df
            print(f"✅ {filename} → {var_name}")

        except Exception as e:
            print(f"❌ {filename} 읽기 실패: {e}")

    return dataframes


def create_individual_variables(dataframes: Dict[str, pd.DataFrame], globals_dict):
    """
    딕셔너리로 저장된 DataFrame들을 개별 변수로 생성

    Parameters:
    -----------
    dataframes : Dict[str, pd.DataFrame]
        load_all_csvs_from_github_folder()의 반환값
    globals_dict : dict
        변수를 저장할 전역 네임스페이스 (globals()를 전달)
    """

    for var_name, df in dataframes.items():
        globals_dict[var_name] = df
        print(f"변수 생성: {var_name}")


# # 사용 예시
# def example_usage():
#     """사용 예시"""

#     print("=== GitHub 폴더 자동 CSV 로더 ===\n")

#     print("사용법:")
#     print("""
#     username = "your-username"
#     repo_name = "korean-elections"
#     folder_path = "temp/v1_g/3rd_2002"

#     # 모든 CSV 파일을 자동으로 읽어와서 딕셔너리로 저장
#     election_data = load_all_csvs_from_github_folder(username, repo_name, folder_path)

#     # 방법 1: 딕셔너리로 접근
#     busan_df = election_data['g3_busan']
#     seoul_df = election_data['g3_seoul']

#     # 방법 2: 개별 변수로 생성
#     create_individual_variables(election_data, globals())
#     # 이제 g3_busan, g3_seoul 등을 직접 사용 가능
#     """)

#     print("\n다양한 번호 지원:")
#     print("""
#     # 3회차: temp1_governor_busan_3.csv → g3_busan
#     election_3rd = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/3rd_2002")

#     # 4회차: temp1_governor_seoul_4.csv → g4_seoul
#     election_4th = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/4th_2006")

#     # 5회차: temp1_governor_daegu_5.csv → g5_daegu
#     election_5th = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/5th_2010")
#     """)


# if __name__ == "__main__":
#     example_usage()

# ### 간단한 사용법

# # 한 줄로 모든 CSV 읽기
# username = "your-github-username"
# repo_name = "korean-elections"
# folder_path = "temp/v1_g/3rd_2002"

# election_data = load_all_csvs_from_github_folder(username, repo_name, folder_path)

# # 사용하기
# busan_df = election_data['g3_busan']
# seoul_df = election_data['g3_seoul']

# # 또는 개별 변수로 생성
# create_individual_variables(election_data, globals())
# # 이제 g3_busan, g3_seoul 직접 사용 가능

In [2]:
import pandas as pd

def show_column_info(df):
    """
    데이터프레임의 열 이름과 인덱스를 확인하는 함수
    """
    print("=== 열 이름과 인덱스 ===")
    for idx, col_name in enumerate(df.columns):
        print(f"{idx}: {col_name}")
    print(f"\n총 열 개수: {len(df.columns)}")
    print(f"총 행 개수: {len(df)}")

def rename_columns_by_index(df, index_or_indices, new_name_or_names):
    """
    인덱스를 통해 열 이름을 변경하는 함수

    Parameters:
    - df: pandas DataFrame
    - index_or_indices: 단일 인덱스(int) 또는 인덱스 리스트([int, int, ...])
    - new_name_or_names: 단일 새 이름(str) 또는 새 이름 리스트([str, str, ...])

    Returns:
    - 열 이름이 변경된 DataFrame
    """
    # 단일 값을 리스트로 변환
    if isinstance(index_or_indices, int):
        indices = [index_or_indices]
    else:
        indices = index_or_indices

    if isinstance(new_name_or_names, str):
        new_names = [new_name_or_names]
    else:
        new_names = new_name_or_names

    # 길이 검증
    if len(indices) != len(new_names):
        raise ValueError(f"인덱스 개수({len(indices)})와 새 이름 개수({len(new_names)})가 일치하지 않습니다.")

    # 인덱스 유효성 검사
    for idx in indices:
        if idx < 0 or idx >= len(df.columns):
            raise ValueError(f"유효하지 않은 인덱스: {idx}. 유효 범위: 0 ~ {len(df.columns) - 1}")

    # 데이터프레임 복사
    result_df = df.copy()

    # 열 이름 변경
    rename_dict = {}
    for idx, new_name in zip(indices, new_names):
        old_name = df.columns[idx]
        rename_dict[old_name] = new_name
        print(f"인덱스 {idx}: '{old_name}' -> '{new_name}'")

    result_df = result_df.rename(columns=rename_dict)

    print(f"\n=== 열 이름 변경 완료 ===")
    print(f"변경된 열 개수: {len(rename_dict)}")

    return result_df

def add_sum_column(df, start_idx, end_idx, new_column_name):
    """
    연속된 인덱스 범위의 열들을 합산하여 새로운 열을 추가하는 함수

    Parameters:
    - df: pandas DataFrame
    - start_idx: 시작 인덱스
    - end_idx: 끝 인덱스 (포함)
    - new_column_name: 새로 추가할 열의 이름

    Returns:
    - 새로운 열이 추가된 DataFrame
    """
    # 유효성 검사
    if start_idx < 0 or end_idx >= len(df.columns) or start_idx > end_idx:
        raise ValueError(f"유효하지 않은 인덱스 범위: ({start_idx}, {end_idx}). 유효 범위: 0 ~ {len(df.columns) - 1}")

    # 합산할 열 이름들 추출
    columns_to_sum = df.columns[start_idx:end_idx + 1]

    print(f"\n=== 합산할 열들 (인덱스 {start_idx}~{end_idx}) ===")
    for idx, col_name in enumerate(columns_to_sum):
        print(f"{start_idx + idx}: {col_name}")

    # 데이터프레임 복사
    result_df = df.copy()

    # 지정된 열들을 합산하여 새로운 열 추가
    # 숫자가 아닌 값은 0으로 처리
    result_df[new_column_name] = df[columns_to_sum].select_dtypes(include=['number']).sum(axis=1, skipna=True)

    print(f"\n=== 새로운 열 추가 완료 ===")
    print(f"새로운 열 이름: {new_column_name}")
    print(f"총 열 개수: {len(result_df.columns)}")

    return result_df

def add_zero_column(df, column_name):
    """
    DataFrame 마지막에 모든 원소가 0(int)인 열을 추가하는 함수

    Parameters:
    - df: pandas DataFrame
    - column_name: 추가할 열의 이름 (str)

    Returns:
    - 새로운 열이 추가된 DataFrame
    """
    result_df = df.copy()
    result_df[column_name] = 0
    return result_df

# # 사용 예시
# if __name__ == "__main__":
#     # CSV 파일 읽기
#     df = pd.read_csv('temp1_president_14.csv')

#     # 열 정보 확인
#     show_column_info(df)

#     # 예시 1: 단일 열 이름 변경
#     print("\n" + "="*50)
#     print("예시 1: 단일 열 이름 변경")
#     print("="*50)
#     df_renamed = rename_columns_by_index(df, 0, '광역시도')

#     # 예시 2: 복수 열 이름 변경
#     print("\n" + "="*50)
#     print("예시 2: 복수 열 이름 변경")
#     print("="*50)
#     df_renamed = rename_columns_by_index(df_renamed, [1, 2], ['시군구', '유권자수'])

#     # 예시 3: 인덱스 4~7 열들 합산 (득표수 1~4번 후보)
#     print("\n" + "="*50)
#     print("예시 3: 열 합산")
#     print("="*50)
#     result_df = add_sum_column(df_renamed, 4, 7, '상위4후보합계')

#     # 변경된 열 정보 확인
#     print("\n" + "="*50)
#     print("변경된 열 정보")
#     print("="*50)
#     show_column_info(result_df)

#     # 결과 확인 (첫 번째 행)
#     print("\n=== 첫 번째 행 결과 ===")
#     first_row = result_df.iloc[0]
#     print(f"광역시도: {first_row['광역시도']}")
#     print(f"시군구: {first_row['시군구']}")
#     print(f"유권자수: {first_row['유권자수']}")
#     print(f"득표수_1_민주자유당_김영삼: {first_row['득표수_1_민주자유당_김영삼']}")
#     print(f"득표수_2_민주당_김대중: {first_row['득표수_2_민주당_김대중']}")
#     print(f"득표수_3_통일국민당_정주영: {first_row['득표수_3_통일국민당_정주영']}")
#     print(f"득표수_4_새한국당_이종찬: {first_row['득표수_4_새한국당_이종찬']}")
#     print(f"상위4후보합계: {first_row['상위4후보합계']}")

#     # 검증: 수동 계산과 비교
#     manual_sum = (first_row['득표수_1_민주자유당_김영삼'] +
#                   first_row['득표수_2_민주당_김대중'] +
#                   first_row['득표수_3_통일국민당_정주영'] +
#                   first_row['득표수_4_새한국당_이종찬'])

#     print(f"\n수동 계산 결과: {manual_sum}")
#     print(f"함수 계산 결과: {first_row['상위4후보합계']}")
#     print(f"계산 일치: {manual_sum == first_row['상위4후보합계']}")

#     # 결과 데이터프레임 출력 (상위 5행)
#     print("\n=== 결과 데이터프레임 (상위 5행) ===")
#     print(result_df.head())

# Data (v1, 7th_2018)

In [3]:
# 한 줄로 모든 CSV 읽기
username = "sw1kwon"
repo_name = "korean-elections"
folder_path = "temp/v1_g/7th_2018"

governor_7 = load_all_csvs_from_github_folder(username, repo_name, folder_path)

# 개별 변수로 생성
create_individual_variables(governor_7, globals())
# 이제 g7_busan, g7_seoul 직접 사용 가능

✅ temp1_governor_busan_7.csv → g7_busan
✅ temp1_governor_chungbuk_7.csv → g7_chungbuk
✅ temp1_governor_chungnam_7.csv → g7_chungnam
✅ temp1_governor_daegu_7.csv → g7_daegu
✅ temp1_governor_daejeon_7.csv → g7_daejeon
✅ temp1_governor_gangwon_7.csv → g7_gangwon
✅ temp1_governor_gwangju_7.csv → g7_gwangju
✅ temp1_governor_gyeongbuk_7.csv → g7_gyeongbuk
✅ temp1_governor_gyeonggi_7.csv → g7_gyeonggi
✅ temp1_governor_gyeongnam_7.csv → g7_gyeongnam
✅ temp1_governor_incheon_7.csv → g7_incheon
✅ temp1_governor_jeju_7.csv → g7_jeju
✅ temp1_governor_jeonbuk_7.csv → g7_jeonbuk
✅ temp1_governor_jeonnam_7.csv → g7_jeonnam
✅ temp1_governor_sejong_7.csv → g7_sejong
✅ temp1_governor_seoul_7.csv → g7_seoul
✅ temp1_governor_ulsan_7.csv → g7_ulsan
변수 생성: g7_busan
변수 생성: g7_chungbuk
변수 생성: g7_chungnam
변수 생성: g7_daegu
변수 생성: g7_daejeon
변수 생성: g7_gangwon
변수 생성: g7_gwangju
변수 생성: g7_gyeongbuk
변수 생성: g7_gyeonggi
변수 생성: g7_gyeongnam
변수 생성: g7_incheon
변수 생성: g7_jeju
변수 생성: g7_jeonbuk
변수 생성: g7_jeonnam
변수 생성: g7_

## Seoul

In [4]:
g7_seoul.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_박원순,득표수_2_자유한국당_김문수,득표수_3_바른미래당_안철수,득표수_5_정의당_김종민,득표수_6_민중당_김진숙,득표수_7_대한애국당_인지연,득표수_8_녹색당_신지예,득표수_9_미래당_우인철,득표수_10_친박연대_최태현,득표수_계,무효투표수,기권수
0,서울특별시,합계,8380947,5019098,2619497,1158487,970374,81664,22134,11222,82874,11599,4021,4961872,57226,3361849
1,서울특별시,종로구,134963,81195,41148,18777,15946,1469,358,223,1896,215,59,80091,1104,53768
2,서울특별시,중구,112336,66474,34243,15618,12814,1023,279,193,1076,149,96,65491,983,45862
3,서울특별시,용산구,200009,116054,53964,31073,24074,2151,436,231,2392,226,115,114662,1392,83955
4,서울특별시,성동구,264634,157863,84106,34878,30379,2594,689,309,2418,370,123,155866,1997,106771


In [5]:
show_column_info(g7_seoul)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_박원순
5: 득표수_2_자유한국당_김문수
6: 득표수_3_바른미래당_안철수
7: 득표수_5_정의당_김종민
8: 득표수_6_민중당_김진숙
9: 득표수_7_대한애국당_인지연
10: 득표수_8_녹색당_신지예
11: 득표수_9_미래당_우인철
12: 득표수_10_친박연대_최태현
13: 득표수_계
14: 무효투표수
15: 기권수

총 열 개수: 16
총 행 개수: 26


### v2.1

In [6]:
g7_seoul_21 = rename_columns_by_index(g7_seoul, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_박원순' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_김문수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [7]:
g7_seoul_21 = add_sum_column(g7_seoul_21, 6, 12, '그외정당')


=== 합산할 열들 (인덱스 6~12) ===
6: 득표수_3_바른미래당_안철수
7: 득표수_5_정의당_김종민
8: 득표수_6_민중당_김진숙
9: 득표수_7_대한애국당_인지연
10: 득표수_8_녹색당_신지예
11: 득표수_9_미래당_우인철
12: 득표수_10_친박연대_최태현

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 17


In [8]:
g7_seoul_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_안철수,득표수_5_정의당_김종민,득표수_6_민중당_김진숙,득표수_7_대한애국당_인지연,득표수_8_녹색당_신지예,득표수_9_미래당_우인철,득표수_10_친박연대_최태현,득표수_계,무효투표수,기권수,그외정당
0,서울특별시,합계,8380947,5019098,2619497,1158487,970374,81664,22134,11222,82874,11599,4021,4961872,57226,3361849,1183888
1,서울특별시,종로구,134963,81195,41148,18777,15946,1469,358,223,1896,215,59,80091,1104,53768,20166
2,서울특별시,중구,112336,66474,34243,15618,12814,1023,279,193,1076,149,96,65491,983,45862,15630


In [9]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_seoul_21 = g7_seoul_21[selected_columns].copy()

g7_seoul_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,서울특별시,합계,8380947,5019098,1158487,2619497,1183888,4961872,57226,3361849
1,서울특별시,종로구,134963,81195,18777,41148,20166,80091,1104,53768
2,서울특별시,중구,112336,66474,15618,34243,15630,65491,983,45862


In [10]:
g7_seoul_21.to_csv("temp2_1_governor_seoul_7.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [11]:
g7_seoul_22 = rename_columns_by_index(g7_seoul, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_박원순' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_김문수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [12]:
g7_seoul_22 = add_sum_column(g7_seoul_22, 6, 12, '그외정당')
g7_seoul_22 = add_zero_column(g7_seoul_22, '무소속')


=== 합산할 열들 (인덱스 6~12) ===
6: 득표수_3_바른미래당_안철수
7: 득표수_5_정의당_김종민
8: 득표수_6_민중당_김진숙
9: 득표수_7_대한애국당_인지연
10: 득표수_8_녹색당_신지예
11: 득표수_9_미래당_우인철
12: 득표수_10_친박연대_최태현

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 17


In [13]:
g7_seoul_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_안철수,득표수_5_정의당_김종민,득표수_6_민중당_김진숙,득표수_7_대한애국당_인지연,득표수_8_녹색당_신지예,득표수_9_미래당_우인철,득표수_10_친박연대_최태현,득표수_계,무효투표수,기권수,그외정당,무소속
0,서울특별시,합계,8380947,5019098,2619497,1158487,970374,81664,22134,11222,82874,11599,4021,4961872,57226,3361849,1183888,0
1,서울특별시,종로구,134963,81195,41148,18777,15946,1469,358,223,1896,215,59,80091,1104,53768,20166,0
2,서울특별시,중구,112336,66474,34243,15618,12814,1023,279,193,1076,149,96,65491,983,45862,15630,0


In [14]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_seoul_22 = g7_seoul_22[selected_columns].copy()

g7_seoul_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,서울특별시,합계,8380947,5019098,1158487,2619497,1183888,0,4961872,57226,3361849
1,서울특별시,종로구,134963,81195,18777,41148,20166,0,80091,1104,53768
2,서울특별시,중구,112336,66474,15618,34243,15630,0,65491,983,45862


In [15]:
g7_seoul_22.to_csv("temp2_2_governor_seoul_7.csv", index=False, encoding="utf-8-sig")

## Busan

In [16]:
g7_busan.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_오거돈,득표수_2_자유한국당_서병수,득표수_3_바른미래당_이성권,득표수_5_정의당_박주미,득표수_6_무소속_이종혁,득표수_계,무효투표수,기권수
0,부산광역시,합계,2939046,1727684,940469,632806,67502,35299,26720,1702796,24888,1211362
1,부산광역시,중구,39832,22499,11730,8993,657,423,300,22103,396,17333
2,부산광역시,서구,97123,55377,27919,22760,1670,1091,913,54353,1024,41746
3,부산광역시,동구,79091,46288,23476,18841,1480,833,691,45321,967,32803
4,부산광역시,영도구,107930,61332,34279,22179,1923,1050,879,60310,1022,46598


In [17]:
show_column_info(g7_busan)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_오거돈
5: 득표수_2_자유한국당_서병수
6: 득표수_3_바른미래당_이성권
7: 득표수_5_정의당_박주미
8: 득표수_6_무소속_이종혁
9: 득표수_계
10: 무효투표수
11: 기권수

총 열 개수: 12
총 행 개수: 17


### v2.1

In [18]:
g7_busan_21 = rename_columns_by_index(g7_busan, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_오거돈' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_서병수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [19]:
g7_busan_21 = add_sum_column(g7_busan_21, 6, 8, '그외정당')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_바른미래당_이성권
7: 득표수_5_정의당_박주미
8: 득표수_6_무소속_이종혁

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [20]:
g7_busan_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_이성권,득표수_5_정의당_박주미,득표수_6_무소속_이종혁,득표수_계,무효투표수,기권수,그외정당
0,부산광역시,합계,2939046,1727684,940469,632806,67502,35299,26720,1702796,24888,1211362,129521
1,부산광역시,중구,39832,22499,11730,8993,657,423,300,22103,396,17333,1380
2,부산광역시,서구,97123,55377,27919,22760,1670,1091,913,54353,1024,41746,3674


In [21]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_busan_21 = g7_busan_21[selected_columns].copy()

g7_busan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,부산광역시,합계,2939046,1727684,632806,940469,129521,1702796,24888,1211362
1,부산광역시,중구,39832,22499,8993,11730,1380,22103,396,17333
2,부산광역시,서구,97123,55377,22760,27919,3674,54353,1024,41746


In [22]:
g7_busan_21.to_csv("temp2_1_governor_busan_7.csv", index=False, encoding="utf-8-sig")

### v2.2

In [23]:
g7_busan_22 = rename_columns_by_index(g7_busan, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_오거돈' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_서병수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [24]:
g7_busan_22 = add_sum_column(g7_busan_22, 6, 7, '그외정당')
g7_busan_22 = add_sum_column(g7_busan_22, 8, 8, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_바른미래당_이성권
7: 득표수_5_정의당_박주미

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13

=== 합산할 열들 (인덱스 8~8) ===
8: 득표수_6_무소속_이종혁

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 14


In [25]:
g7_busan_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_이성권,득표수_5_정의당_박주미,득표수_6_무소속_이종혁,득표수_계,무효투표수,기권수,그외정당,무소속
0,부산광역시,합계,2939046,1727684,940469,632806,67502,35299,26720,1702796,24888,1211362,102801,26720
1,부산광역시,중구,39832,22499,11730,8993,657,423,300,22103,396,17333,1080,300
2,부산광역시,서구,97123,55377,27919,22760,1670,1091,913,54353,1024,41746,2761,913


In [26]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_busan_22 = g7_busan_22[selected_columns].copy()

g7_busan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,부산광역시,합계,2939046,1727684,632806,940469,102801,26720,1702796,24888,1211362
1,부산광역시,중구,39832,22499,8993,11730,1080,300,22103,396,17333
2,부산광역시,서구,97123,55377,22760,27919,2761,913,54353,1024,41746


In [27]:
g7_busan_22.to_csv("temp2_2_governor_busan_7.csv", index=False, encoding="utf-8-sig")

## Daegu

In [28]:
g7_daegu.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_임대윤,득표수_2_자유한국당_권영진,득표수_3_바른미래당_김형기,득표수_계,무효투표수,기권수
0,대구광역시,합계,2047286,1172279,458112,619165,74955,1152232,20047,875007
1,대구광역시,중구,68303,39863,14983,21640,2572,39195,668,28440
2,대구광역시,동구,295860,171627,65465,90378,12894,168737,2890,124233
3,대구광역시,서구,167260,89634,28180,53879,5532,87591,2043,77626
4,대구광역시,남구,134493,72239,25478,41302,3886,70666,1573,62254


In [29]:
show_column_info(g7_daegu)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_임대윤
5: 득표수_2_자유한국당_권영진
6: 득표수_3_바른미래당_김형기
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 9


### v2.1

In [30]:
g7_daegu_21 = rename_columns_by_index(g7_daegu, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_임대윤' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_권영진' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [31]:
g7_daegu_21 = add_sum_column(g7_daegu_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_바른미래당_김형기

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [32]:
g7_daegu_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_김형기,득표수_계,무효투표수,기권수,그외정당
0,대구광역시,합계,2047286,1172279,458112,619165,74955,1152232,20047,875007,74955
1,대구광역시,중구,68303,39863,14983,21640,2572,39195,668,28440,2572
2,대구광역시,동구,295860,171627,65465,90378,12894,168737,2890,124233,12894


In [33]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_daegu_21 = g7_daegu_21[selected_columns].copy()

g7_daegu_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,대구광역시,합계,2047286,1172279,619165,458112,74955,1152232,20047,875007
1,대구광역시,중구,68303,39863,21640,14983,2572,39195,668,28440
2,대구광역시,동구,295860,171627,90378,65465,12894,168737,2890,124233


In [34]:
g7_daegu_21.to_csv("temp2_1_governor_daegu_7.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [35]:
g7_daegu_22 = rename_columns_by_index(g7_daegu, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_임대윤' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_권영진' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [36]:
g7_daegu_22 = add_sum_column(g7_daegu_22, 6, 6, '그외정당')
g7_daegu_22 = add_zero_column(g7_daegu_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_바른미래당_김형기

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [37]:
g7_daegu_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_김형기,득표수_계,무효투표수,기권수,그외정당,무소속
0,대구광역시,합계,2047286,1172279,458112,619165,74955,1152232,20047,875007,74955,0
1,대구광역시,중구,68303,39863,14983,21640,2572,39195,668,28440,2572,0
2,대구광역시,동구,295860,171627,65465,90378,12894,168737,2890,124233,12894,0


In [38]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_daegu_22 = g7_daegu_22[selected_columns].copy()

g7_daegu_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,대구광역시,합계,2047286,1172279,619165,458112,74955,0,1152232,20047,875007
1,대구광역시,중구,68303,39863,21640,14983,2572,0,39195,668,28440
2,대구광역시,동구,295860,171627,90378,65465,12894,0,168737,2890,124233


In [39]:
g7_daegu_22.to_csv("temp2_2_governor_daegu_7.csv", index=False, encoding="utf-8-sig")

## Incheon

In [40]:
g7_incheon.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_박남춘,득표수_2_자유한국당_유정복,득표수_3_바른미래당_문병호,득표수_5_정의당_김응호,득표수_계,무효투표수,기권수
0,인천광역시,합계,2440779,1349084,766186,470937,54054,37472,1328649,20435,1091695
1,인천광역시,중구,98608,53666,29817,19724,1960,1301,52802,864,44942
2,인천광역시,동구,57559,34757,17288,14655,895,1175,34013,744,22802
3,인천광역시,남구,356508,183835,97347,71278,6754,5318,180697,3138,172673
4,인천광역시,연수구,269626,160548,90969,58191,5277,4153,158590,1958,109078


In [41]:
show_column_info(g7_incheon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_박남춘
5: 득표수_2_자유한국당_유정복
6: 득표수_3_바른미래당_문병호
7: 득표수_5_정의당_김응호
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 11


### v2.1

In [42]:
g7_incheon_21 = rename_columns_by_index(g7_incheon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_박남춘' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_유정복' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [43]:
g7_incheon_21 = add_sum_column(g7_incheon_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_바른미래당_문병호
7: 득표수_5_정의당_김응호

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [44]:
g7_incheon_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_문병호,득표수_5_정의당_김응호,득표수_계,무효투표수,기권수,그외정당
0,인천광역시,합계,2440779,1349084,766186,470937,54054,37472,1328649,20435,1091695,91526
1,인천광역시,중구,98608,53666,29817,19724,1960,1301,52802,864,44942,3261
2,인천광역시,동구,57559,34757,17288,14655,895,1175,34013,744,22802,2070


In [45]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_incheon_21 = g7_incheon_21[selected_columns].copy()

g7_incheon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,인천광역시,합계,2440779,1349084,470937,766186,91526,1328649,20435,1091695
1,인천광역시,중구,98608,53666,19724,29817,3261,52802,864,44942
2,인천광역시,동구,57559,34757,14655,17288,2070,34013,744,22802


In [46]:
g7_incheon_21.to_csv("temp2_1_governor_incheon_7.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [47]:
g7_incheon_22 = rename_columns_by_index(g7_incheon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_박남춘' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_유정복' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [48]:
g7_incheon_22 = add_sum_column(g7_incheon_22, 6, 7, '그외정당')
g7_incheon_22 = add_zero_column(g7_incheon_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_바른미래당_문병호
7: 득표수_5_정의당_김응호

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [49]:
g7_incheon_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_문병호,득표수_5_정의당_김응호,득표수_계,무효투표수,기권수,그외정당,무소속
0,인천광역시,합계,2440779,1349084,766186,470937,54054,37472,1328649,20435,1091695,91526,0
1,인천광역시,중구,98608,53666,29817,19724,1960,1301,52802,864,44942,3261,0
2,인천광역시,동구,57559,34757,17288,14655,895,1175,34013,744,22802,2070,0


In [50]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_incheon_22 = g7_incheon_22[selected_columns].copy()

g7_incheon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,인천광역시,합계,2440779,1349084,470937,766186,91526,0,1328649,20435,1091695
1,인천광역시,중구,98608,53666,19724,29817,3261,0,52802,864,44942
2,인천광역시,동구,57559,34757,14655,17288,2070,0,34013,744,22802


In [51]:
g7_incheon_22.to_csv("temp2_2_governor_incheon_7.csv", index=False, encoding="utf-8-sig")

## Gwangju

In [52]:
g7_gwangju.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이용섭,득표수_3_바른미래당_전덕영,득표수_5_정의당_나경채,득표수_6_민중당_윤민호,득표수_계,무효투표수,기권수
0,광주광역시,합계,1172429,694252,573995,34487,40916,33312,682710,11542,478177
1,광주광역시,동구,82265,51550,42139,3489,2688,2101,50417,1133,30715
2,광주광역시,서구,247275,147798,122803,7163,8813,6765,145544,2254,99477
3,광주광역시,남구,176964,106294,88699,5569,6042,4143,104453,1841,70670
4,광주광역시,북구,358914,210587,173766,9854,11694,11693,207007,3580,148327


In [53]:
show_column_info(g7_gwangju)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_이용섭
5: 득표수_3_바른미래당_전덕영
6: 득표수_5_정의당_나경채
7: 득표수_6_민중당_윤민호
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 6


### v2.1

In [54]:
g7_gwangju_21 = rename_columns_by_index(g7_gwangju, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_이용섭' -> '진보정당'
인덱스 5: '득표수_3_바른미래당_전덕영' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [55]:
g7_gwangju_21 = add_sum_column(g7_gwangju_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_5_정의당_나경채
7: 득표수_6_민중당_윤민호

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [56]:
g7_gwangju_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_5_정의당_나경채,득표수_6_민중당_윤민호,득표수_계,무효투표수,기권수,그외정당
0,광주광역시,합계,1172429,694252,573995,34487,40916,33312,682710,11542,478177,74228
1,광주광역시,동구,82265,51550,42139,3489,2688,2101,50417,1133,30715,4789
2,광주광역시,서구,247275,147798,122803,7163,8813,6765,145544,2254,99477,15578


In [57]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_gwangju_21 = g7_gwangju_21[selected_columns].copy()

g7_gwangju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,광주광역시,합계,1172429,694252,34487,573995,74228,682710,11542,478177
1,광주광역시,동구,82265,51550,3489,42139,4789,50417,1133,30715
2,광주광역시,서구,247275,147798,7163,122803,15578,145544,2254,99477


In [58]:
g7_gwangju_21.to_csv("temp2_1_governor_gwangju_7.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [59]:
g7_gwangju_22 = rename_columns_by_index(g7_gwangju, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_이용섭' -> '진보정당'
인덱스 5: '득표수_3_바른미래당_전덕영' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [60]:
g7_gwangju_22 = add_sum_column(g7_gwangju_22, 6, 7, '그외정당')
g7_gwangju_22 = add_zero_column(g7_gwangju_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_5_정의당_나경채
7: 득표수_6_민중당_윤민호

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [61]:
g7_gwangju_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_5_정의당_나경채,득표수_6_민중당_윤민호,득표수_계,무효투표수,기권수,그외정당,무소속
0,광주광역시,합계,1172429,694252,573995,34487,40916,33312,682710,11542,478177,74228,0
1,광주광역시,동구,82265,51550,42139,3489,2688,2101,50417,1133,30715,4789,0
2,광주광역시,서구,247275,147798,122803,7163,8813,6765,145544,2254,99477,15578,0


In [62]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_gwangju_22 = g7_gwangju_22[selected_columns].copy()

g7_gwangju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,광주광역시,합계,1172429,694252,34487,573995,74228,0,682710,11542,478177
1,광주광역시,동구,82265,51550,3489,42139,4789,0,50417,1133,30715
2,광주광역시,서구,247275,147798,7163,122803,15578,0,145544,2254,99477


In [63]:
g7_gwangju_22.to_csv("temp2_2_governor_gwangju_7.csv", index=False, encoding="utf-8-sig")

## Daejeon

In [64]:
g7_daejeon.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_허태정,득표수_2_자유한국당_박성효,득표수_3_바른미래당_남충희,득표수_5_정의당_김윤기,득표수_계,무효투표수,기권수
0,대전광역시,합계,1219513,706983,393354,224306,61271,18351,697282,9701,512530
1,대전광역시,동구,191362,106341,55928,35880,9750,2755,104313,2028,85021
2,대전광역시,중구,206042,116950,60878,41632,9863,2730,115103,1847,89092
3,대전광역시,서구,395367,226532,127151,71325,19262,6140,223878,2654,168835
4,대전광역시,유성구,272663,168310,101687,43981,16466,4571,166705,1605,104353


In [65]:
show_column_info(g7_daejeon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_허태정
5: 득표수_2_자유한국당_박성효
6: 득표수_3_바른미래당_남충희
7: 득표수_5_정의당_김윤기
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 6


### v2.1

In [66]:
g7_daejeon_21 = rename_columns_by_index(g7_daejeon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_허태정' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_박성효' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [67]:
g7_daejeon_21 = add_sum_column(g7_daejeon_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_바른미래당_남충희
7: 득표수_5_정의당_김윤기

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [68]:
g7_daejeon_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_남충희,득표수_5_정의당_김윤기,득표수_계,무효투표수,기권수,그외정당
0,대전광역시,합계,1219513,706983,393354,224306,61271,18351,697282,9701,512530,79622
1,대전광역시,동구,191362,106341,55928,35880,9750,2755,104313,2028,85021,12505
2,대전광역시,중구,206042,116950,60878,41632,9863,2730,115103,1847,89092,12593


In [69]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_daejeon_21 = g7_daejeon_21[selected_columns].copy()

g7_daejeon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,대전광역시,합계,1219513,706983,224306,393354,79622,697282,9701,512530
1,대전광역시,동구,191362,106341,35880,55928,12505,104313,2028,85021
2,대전광역시,중구,206042,116950,41632,60878,12593,115103,1847,89092


In [70]:
g7_daejeon_21.to_csv("temp2_1_governor_daejeon_7.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [71]:
g7_daejeon_22 = rename_columns_by_index(g7_daejeon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_허태정' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_박성효' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [72]:
g7_daejeon_22 = add_sum_column(g7_daejeon_22, 6, 7, '그외정당')
g7_daejeon_22 = add_zero_column(g7_daejeon_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_바른미래당_남충희
7: 득표수_5_정의당_김윤기

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [73]:
g7_daejeon_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_남충희,득표수_5_정의당_김윤기,득표수_계,무효투표수,기권수,그외정당,무소속
0,대전광역시,합계,1219513,706983,393354,224306,61271,18351,697282,9701,512530,79622,0
1,대전광역시,동구,191362,106341,55928,35880,9750,2755,104313,2028,85021,12505,0
2,대전광역시,중구,206042,116950,60878,41632,9863,2730,115103,1847,89092,12593,0


In [74]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_daejeon_22 = g7_daejeon_22[selected_columns].copy()

g7_daejeon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,대전광역시,합계,1219513,706983,224306,393354,79622,0,697282,9701,512530
1,대전광역시,동구,191362,106341,35880,55928,12505,0,104313,2028,85021
2,대전광역시,중구,206042,116950,41632,60878,12593,0,115103,1847,89092


In [75]:
g7_daejeon_22.to_csv("temp2_2_governor_daejeon_7.csv", index=False, encoding="utf-8-sig")

## Ulsan

In [76]:
g7_ulsan.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송철호,득표수_2_자유한국당_김기현,득표수_3_바른미래당_이영희,득표수_6_민중당_김창현,득표수_계,무효투표수,기권수
0,울산광역시,합계,942550,610698,317341,240475,13589,28621,600026,10672,331852
1,울산광역시,중구,194908,128337,65014,54227,2553,4078,125872,2465,66571
2,울산광역시,남구,274930,176694,89518,75240,3502,5989,174249,2445,98236
3,울산광역시,동구,134316,86763,47190,28418,2151,7154,84913,1850,47553
4,울산광역시,북구,159030,104132,58639,33758,2506,7943,102846,1286,54898


In [77]:
show_column_info(g7_ulsan)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_송철호
5: 득표수_2_자유한국당_김기현
6: 득표수_3_바른미래당_이영희
7: 득표수_6_민중당_김창현
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 6


### v2.1

In [78]:
g7_ulsan_21 = rename_columns_by_index(g7_ulsan, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_송철호' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_김기현' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [79]:
g7_ulsan_21 = add_sum_column(g7_ulsan_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_바른미래당_이영희
7: 득표수_6_민중당_김창현

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [80]:
g7_ulsan_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_이영희,득표수_6_민중당_김창현,득표수_계,무효투표수,기권수,그외정당
0,울산광역시,합계,942550,610698,317341,240475,13589,28621,600026,10672,331852,42210
1,울산광역시,중구,194908,128337,65014,54227,2553,4078,125872,2465,66571,6631
2,울산광역시,남구,274930,176694,89518,75240,3502,5989,174249,2445,98236,9491


In [81]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_ulsan_21 = g7_ulsan_21[selected_columns].copy()

g7_ulsan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,울산광역시,합계,942550,610698,240475,317341,42210,600026,10672,331852
1,울산광역시,중구,194908,128337,54227,65014,6631,125872,2465,66571
2,울산광역시,남구,274930,176694,75240,89518,9491,174249,2445,98236


In [82]:
g7_ulsan_21.to_csv("temp2_1_governor_ulsan_7.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [83]:
g7_ulsan_22 = rename_columns_by_index(g7_ulsan, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_송철호' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_김기현' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [84]:
g7_ulsan_22 = add_sum_column(g7_ulsan_22, 6, 7, '그외정당')
g7_ulsan_22 = add_zero_column(g7_ulsan_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_바른미래당_이영희
7: 득표수_6_민중당_김창현

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [85]:
g7_ulsan_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_이영희,득표수_6_민중당_김창현,득표수_계,무효투표수,기권수,그외정당,무소속
0,울산광역시,합계,942550,610698,317341,240475,13589,28621,600026,10672,331852,42210,0
1,울산광역시,중구,194908,128337,65014,54227,2553,4078,125872,2465,66571,6631,0
2,울산광역시,남구,274930,176694,89518,75240,3502,5989,174249,2445,98236,9491,0


In [86]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_ulsan_22 = g7_ulsan_22[selected_columns].copy()

g7_ulsan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,울산광역시,합계,942550,610698,240475,317341,42210,0,600026,10672,331852
1,울산광역시,중구,194908,128337,54227,65014,6631,0,125872,2465,66571
2,울산광역시,남구,274930,176694,75240,89518,9491,0,174249,2445,98236


In [87]:
g7_ulsan_22.to_csv("temp2_2_governor_ulsan_7.csv", index=False, encoding="utf-8-sig")

## Sejong

In [88]:
g7_sejong.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이춘희,득표수_2_자유한국당_송아영,득표수_3_바른미래당_허철회,득표수_계,무효투표수,기권수
0,세종특별자치시,합계,222852,137603,96896,24546,14444,135886,1717,85249


In [89]:
show_column_info(g7_sejong)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_이춘희
5: 득표수_2_자유한국당_송아영
6: 득표수_3_바른미래당_허철회
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 1


### v2.1

In [90]:
g7_sejong_21 = rename_columns_by_index(g7_sejong, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_이춘희' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_송아영' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [91]:
g7_sejong_21 = add_sum_column(g7_sejong_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_바른미래당_허철회

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [92]:
g7_sejong_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_허철회,득표수_계,무효투표수,기권수,그외정당
0,세종특별자치시,합계,222852,137603,96896,24546,14444,135886,1717,85249,14444


In [93]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_sejong_21 = g7_sejong_21[selected_columns].copy()

g7_sejong_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,세종특별자치시,합계,222852,137603,24546,96896,14444,135886,1717,85249


In [94]:
g7_sejong_21.to_csv("temp2_1_governor_sejong_7.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [95]:
g7_sejong_22 = rename_columns_by_index(g7_sejong, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_이춘희' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_송아영' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [96]:
g7_sejong_22 = add_sum_column(g7_sejong_22, 6, 6, '그외정당')
g7_sejong_22 = add_zero_column(g7_sejong_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_바른미래당_허철회

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [97]:
g7_sejong_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_허철회,득표수_계,무효투표수,기권수,그외정당,무소속
0,세종특별자치시,합계,222852,137603,96896,24546,14444,135886,1717,85249,14444,0


In [98]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_sejong_22 = g7_sejong_22[selected_columns].copy()

g7_sejong_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,세종특별자치시,합계,222852,137603,24546,96896,14444,0,135886,1717,85249


In [99]:
g7_sejong_22.to_csv("temp2_2_governor_sejong_7.csv", index=False, encoding="utf-8-sig")

## Gyeonggi

In [100]:
g7_gyeonggi.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이재명,득표수_2_자유한국당_남경필,득표수_3_바른미래당_김영환,득표수_5_정의당_이홍우,득표수_6_민중당_홍성규,득표수_계,무효투표수,기권수
0,경기도,합계,10533027,6084955,3370621,2122433,287504,151871,43098,5975527,109428,4448072
1,경기도,수원시장안구,202140,121951,64965,45405,5638,3050,768,119826,2125,80189
2,경기도,수원시권선구,342897,197363,110066,69133,8656,4787,1473,194115,3248,145534
3,경기도,수원시팔달구,161673,88388,44464,36057,3707,1908,552,86688,1700,73285
4,경기도,수원시영통구,273531,174692,98513,59730,8134,4762,873,172012,2680,98839


In [101]:
show_column_info(g7_gyeonggi)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_이재명
5: 득표수_2_자유한국당_남경필
6: 득표수_3_바른미래당_김영환
7: 득표수_5_정의당_이홍우
8: 득표수_6_민중당_홍성규
9: 득표수_계
10: 무효투표수
11: 기권수

총 열 개수: 12
총 행 개수: 43


### v2.1

In [102]:
g7_gyeonggi_21 = rename_columns_by_index(g7_gyeonggi, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_이재명' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_남경필' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [103]:
g7_gyeonggi_21 = add_sum_column(g7_gyeonggi_21, 6, 8, '그외정당')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_바른미래당_김영환
7: 득표수_5_정의당_이홍우
8: 득표수_6_민중당_홍성규

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [104]:
g7_gyeonggi_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_김영환,득표수_5_정의당_이홍우,득표수_6_민중당_홍성규,득표수_계,무효투표수,기권수,그외정당
0,경기도,합계,10533027,6084955,3370621,2122433,287504,151871,43098,5975527,109428,4448072,482473
1,경기도,수원시장안구,202140,121951,64965,45405,5638,3050,768,119826,2125,80189,9456
2,경기도,수원시권선구,342897,197363,110066,69133,8656,4787,1473,194115,3248,145534,14916


In [105]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_gyeonggi_21 = g7_gyeonggi_21[selected_columns].copy()

g7_gyeonggi_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경기도,합계,10533027,6084955,2122433,3370621,482473,5975527,109428,4448072
1,경기도,수원시장안구,202140,121951,45405,64965,9456,119826,2125,80189
2,경기도,수원시권선구,342897,197363,69133,110066,14916,194115,3248,145534


In [106]:
g7_gyeonggi_21.to_csv("temp2_1_governor_gyeonggi_7.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [107]:
g7_gyeonggi_22 = rename_columns_by_index(g7_gyeonggi, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_이재명' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_남경필' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [108]:
g7_gyeonggi_22 = add_sum_column(g7_gyeonggi_22, 6, 8, '그외정당')
g7_gyeonggi_22 = add_zero_column(g7_gyeonggi_22, '무소속')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_바른미래당_김영환
7: 득표수_5_정의당_이홍우
8: 득표수_6_민중당_홍성규

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [109]:
g7_gyeonggi_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_김영환,득표수_5_정의당_이홍우,득표수_6_민중당_홍성규,득표수_계,무효투표수,기권수,그외정당,무소속
0,경기도,합계,10533027,6084955,3370621,2122433,287504,151871,43098,5975527,109428,4448072,482473,0
1,경기도,수원시장안구,202140,121951,64965,45405,5638,3050,768,119826,2125,80189,9456,0
2,경기도,수원시권선구,342897,197363,110066,69133,8656,4787,1473,194115,3248,145534,14916,0


In [110]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_gyeonggi_22 = g7_gyeonggi_22[selected_columns].copy()

g7_gyeonggi_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경기도,합계,10533027,6084955,2122433,3370621,482473,0,5975527,109428,4448072
1,경기도,수원시장안구,202140,121951,45405,64965,9456,0,119826,2125,80189
2,경기도,수원시권선구,342897,197363,69133,110066,14916,0,194115,3248,145534


In [111]:
g7_gyeonggi_22.to_csv("temp2_2_governor_gyeonggi_7.csv", index=False, encoding="utf-8-sig")

## Gangwon

In [112]:
g7_gangwon.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_최문순,득표수_2_자유한국당_정창수,득표수_계,무효투표수,기권수
0,강원도,합계,1296196,819193,518447,282456,800903,18290,477003
1,강원도,춘천시,231430,144282,96699,45024,141723,2559,87148
2,강원도,원주시,278991,159235,103811,53060,156871,2364,119756
3,강원도,강릉시,179767,108772,62434,44086,106520,2252,70995
4,강원도,동해시,75747,46923,29034,16783,45817,1106,28824


In [113]:
show_column_info(g7_gangwon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_최문순
5: 득표수_2_자유한국당_정창수
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 19


### v2.1 (XX)

In [114]:
g7_gangwon_21 = rename_columns_by_index(g7_gangwon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_최문순' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_정창수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [115]:
g7_gangwon_21 = add_zero_column(g7_gangwon_21, '그외정당')

In [116]:
g7_gangwon_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당
0,강원도,합계,1296196,819193,518447,282456,800903,18290,477003,0
1,강원도,춘천시,231430,144282,96699,45024,141723,2559,87148,0
2,강원도,원주시,278991,159235,103811,53060,156871,2364,119756,0


In [117]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_gangwon_21 = g7_gangwon_21[selected_columns].copy()

g7_gangwon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,강원도,합계,1296196,819193,282456,518447,0,800903,18290,477003
1,강원도,춘천시,231430,144282,45024,96699,0,141723,2559,87148
2,강원도,원주시,278991,159235,53060,103811,0,156871,2364,119756


In [118]:
g7_gangwon_21.to_csv("temp2_1_governor_gangwon_7.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX, X)

In [119]:
g7_gangwon_22 = rename_columns_by_index(g7_gangwon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_최문순' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_정창수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [120]:
g7_gangwon_22 = add_zero_column(g7_gangwon_22, '그외정당')
g7_gangwon_22 = add_zero_column(g7_gangwon_22, '무소속')

In [121]:
g7_gangwon_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당,무소속
0,강원도,합계,1296196,819193,518447,282456,800903,18290,477003,0,0
1,강원도,춘천시,231430,144282,96699,45024,141723,2559,87148,0,0
2,강원도,원주시,278991,159235,103811,53060,156871,2364,119756,0,0


In [122]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_gangwon_22 = g7_gangwon_22[selected_columns].copy()

g7_gangwon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,강원도,합계,1296196,819193,282456,518447,0,0,800903,18290,477003
1,강원도,춘천시,231430,144282,45024,96699,0,0,141723,2559,87148
2,강원도,원주시,278991,159235,53060,103811,0,0,156871,2364,119756


In [123]:
g7_gangwon_22.to_csv("temp2_2_governor_gangwon_7.csv", index=False, encoding="utf-8-sig")

## Chungbuk

In [124]:
g7_chungbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이시종,득표수_2_자유한국당_박경국,득표수_3_바른미래당_신용한,득표수_계,무효투표수,기권수
0,충청북도,합계,1318186,782316,468750,227371,70330,766451,15865,535870
1,충청북도,청주시상당구,140405,80162,47260,23920,7676,78856,1306,60243
2,충청북도,청주시서원구,174307,100218,60096,28299,10359,98754,1464,74089
3,충청북도,청주시흥덕구,204639,110108,67695,28114,12684,108493,1615,94531
4,충청북도,청주시청원구,154249,85634,54139,21312,8988,84439,1195,68615


In [125]:
show_column_info(g7_chungbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_이시종
5: 득표수_2_자유한국당_박경국
6: 득표수_3_바른미래당_신용한
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 15


### v2.1

In [126]:
g7_chungbuk_21 = rename_columns_by_index(g7_chungbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_이시종' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_박경국' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [127]:
g7_chungbuk_21 = add_sum_column(g7_chungbuk_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_바른미래당_신용한

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [128]:
g7_chungbuk_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_신용한,득표수_계,무효투표수,기권수,그외정당
0,충청북도,합계,1318186,782316,468750,227371,70330,766451,15865,535870,70330
1,충청북도,청주시상당구,140405,80162,47260,23920,7676,78856,1306,60243,7676
2,충청북도,청주시서원구,174307,100218,60096,28299,10359,98754,1464,74089,10359


In [129]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_chungbuk_21 = g7_chungbuk_21[selected_columns].copy()

g7_chungbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,충청북도,합계,1318186,782316,227371,468750,70330,766451,15865,535870
1,충청북도,청주시상당구,140405,80162,23920,47260,7676,78856,1306,60243
2,충청북도,청주시서원구,174307,100218,28299,60096,10359,98754,1464,74089


In [130]:
g7_chungbuk_21.to_csv("temp2_1_governor_chungbuk_7.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [131]:
g7_chungbuk_22 = rename_columns_by_index(g7_chungbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_이시종' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_박경국' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [132]:
g7_chungbuk_22 = add_sum_column(g7_chungbuk_22, 6, 6, '그외정당')
g7_chungbuk_22 = add_zero_column(g7_chungbuk_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_바른미래당_신용한

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [133]:
g7_chungbuk_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_신용한,득표수_계,무효투표수,기권수,그외정당,무소속
0,충청북도,합계,1318186,782316,468750,227371,70330,766451,15865,535870,70330,0
1,충청북도,청주시상당구,140405,80162,47260,23920,7676,78856,1306,60243,7676,0
2,충청북도,청주시서원구,174307,100218,60096,28299,10359,98754,1464,74089,10359,0


In [134]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_chungbuk_22 = g7_chungbuk_22[selected_columns].copy()

g7_chungbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,충청북도,합계,1318186,782316,227371,468750,70330,0,766451,15865,535870
1,충청북도,청주시상당구,140405,80162,23920,47260,7676,0,78856,1306,60243
2,충청북도,청주시서원구,174307,100218,28299,60096,10359,0,98754,1464,74089


In [135]:
g7_chungbuk_22.to_csv("temp2_2_governor_chungbuk_7.csv", index=False, encoding="utf-8-sig")

## Chungnam

In [136]:
g7_chungnam.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_양승조,득표수_2_자유한국당_이인제,득표수_6_가자코리아_차국환,득표수_계,무효투표수,기권수
0,충청남도,합계,1740413,1011447,615870,345577,23012,984459,26988,728966
1,충청남도,천안시서북구,298093,159458,117098,37427,2705,157230,2228,138635
2,충청남도,천안시동남구,211724,106417,69999,32706,1773,104478,1939,105307
3,충청남도,공주시,92935,56756,32526,21453,1128,55107,1649,36179
4,충청남도,보령시,87281,56352,28981,23856,1384,54221,2131,30929


In [137]:
show_column_info(g7_chungnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_양승조
5: 득표수_2_자유한국당_이인제
6: 득표수_6_가자코리아_차국환
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 17


### v2.1

In [138]:
g7_chungnam_21 = rename_columns_by_index(g7_chungnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_양승조' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_이인제' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [139]:
g7_chungnam_21 = add_sum_column(g7_chungnam_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_6_가자코리아_차국환

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [140]:
g7_chungnam_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_6_가자코리아_차국환,득표수_계,무효투표수,기권수,그외정당
0,충청남도,합계,1740413,1011447,615870,345577,23012,984459,26988,728966,23012
1,충청남도,천안시서북구,298093,159458,117098,37427,2705,157230,2228,138635,2705
2,충청남도,천안시동남구,211724,106417,69999,32706,1773,104478,1939,105307,1773


In [141]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_chungnam_21 = g7_chungnam_21[selected_columns].copy()

g7_chungnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,충청남도,합계,1740413,1011447,345577,615870,23012,984459,26988,728966
1,충청남도,천안시서북구,298093,159458,37427,117098,2705,157230,2228,138635
2,충청남도,천안시동남구,211724,106417,32706,69999,1773,104478,1939,105307


In [142]:
g7_chungnam_21.to_csv("temp2_1_governor_chungnam_7.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [143]:
g7_chungnam_22 = rename_columns_by_index(g7_chungnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_양승조' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_이인제' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [144]:
g7_chungnam_22 = add_sum_column(g7_chungnam_22, 6, 6, '그외정당')
g7_chungnam_22 = add_zero_column(g7_chungnam_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_6_가자코리아_차국환

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [145]:
g7_chungnam_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_6_가자코리아_차국환,득표수_계,무효투표수,기권수,그외정당,무소속
0,충청남도,합계,1740413,1011447,615870,345577,23012,984459,26988,728966,23012,0
1,충청남도,천안시서북구,298093,159458,117098,37427,2705,157230,2228,138635,2705,0
2,충청남도,천안시동남구,211724,106417,69999,32706,1773,104478,1939,105307,1773,0


In [146]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_chungnam_22 = g7_chungnam_22[selected_columns].copy()

g7_chungnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,충청남도,합계,1740413,1011447,345577,615870,23012,0,984459,26988,728966
1,충청남도,천안시서북구,298093,159458,37427,117098,2705,0,157230,2228,138635
2,충청남도,천안시동남구,211724,106417,32706,69999,1773,0,104478,1939,105307


In [147]:
g7_chungnam_22.to_csv("temp2_2_governor_chungnam_7.csv", index=False, encoding="utf-8-sig")

## Jeonbuk

In [148]:
g7_jeonbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송하진,득표수_2_자유한국당_신재봉,득표수_4_민주평화당_임정엽,득표수_5_정의당_권태홍,득표수_6_민중당_이광석,득표수_계,무효투표수,기권수
0,전라북도,합계,1527729,996504,682042,26374,184728,52496,20827,966467,30037,531225
1,전라북도,전주시완산구,313929,193303,121422,4205,48517,13719,2092,189955,3348,120626
2,전라북도,전주시덕진구,210274,130716,80531,2713,34008,9616,1721,128589,2127,79558
3,전라북도,군산시,223434,134447,98640,4475,17447,6656,3205,130423,4024,88987
4,전라북도,익산시,245566,153275,111862,3844,22976,7986,2244,148912,4363,92291


In [149]:
show_column_info(g7_jeonbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_송하진
5: 득표수_2_자유한국당_신재봉
6: 득표수_4_민주평화당_임정엽
7: 득표수_5_정의당_권태홍
8: 득표수_6_민중당_이광석
9: 득표수_계
10: 무효투표수
11: 기권수

총 열 개수: 12
총 행 개수: 16


### v2.1

In [150]:
g7_jeonbuk_21 = rename_columns_by_index(g7_jeonbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_송하진' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_신재봉' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [151]:
g7_jeonbuk_21 = add_sum_column(g7_jeonbuk_21, 6, 8, '그외정당')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_4_민주평화당_임정엽
7: 득표수_5_정의당_권태홍
8: 득표수_6_민중당_이광석

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [152]:
g7_jeonbuk_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주평화당_임정엽,득표수_5_정의당_권태홍,득표수_6_민중당_이광석,득표수_계,무효투표수,기권수,그외정당
0,전라북도,합계,1527729,996504,682042,26374,184728,52496,20827,966467,30037,531225,258051
1,전라북도,전주시완산구,313929,193303,121422,4205,48517,13719,2092,189955,3348,120626,64328
2,전라북도,전주시덕진구,210274,130716,80531,2713,34008,9616,1721,128589,2127,79558,45345


In [153]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_jeonbuk_21 = g7_jeonbuk_21[selected_columns].copy()

g7_jeonbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,전라북도,합계,1527729,996504,26374,682042,258051,966467,30037,531225
1,전라북도,전주시완산구,313929,193303,4205,121422,64328,189955,3348,120626
2,전라북도,전주시덕진구,210274,130716,2713,80531,45345,128589,2127,79558


In [154]:
g7_jeonbuk_21.to_csv("temp2_1_governor_jeonbuk_7.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [155]:
g7_jeonbuk_22 = rename_columns_by_index(g7_jeonbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_송하진' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_신재봉' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [156]:
g7_jeonbuk_22 = add_sum_column(g7_jeonbuk_22, 6, 8, '그외정당')
g7_jeonbuk_22 = add_zero_column(g7_jeonbuk_22, '무소속')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_4_민주평화당_임정엽
7: 득표수_5_정의당_권태홍
8: 득표수_6_민중당_이광석

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [157]:
g7_jeonbuk_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주평화당_임정엽,득표수_5_정의당_권태홍,득표수_6_민중당_이광석,득표수_계,무효투표수,기권수,그외정당,무소속
0,전라북도,합계,1527729,996504,682042,26374,184728,52496,20827,966467,30037,531225,258051,0
1,전라북도,전주시완산구,313929,193303,121422,4205,48517,13719,2092,189955,3348,120626,64328,0
2,전라북도,전주시덕진구,210274,130716,80531,2713,34008,9616,1721,128589,2127,79558,45345,0


In [158]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_jeonbuk_22 = g7_jeonbuk_22[selected_columns].copy()

g7_jeonbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,전라북도,합계,1527729,996504,26374,682042,258051,0,966467,30037,531225
1,전라북도,전주시완산구,313929,193303,4205,121422,64328,0,189955,3348,120626
2,전라북도,전주시덕진구,210274,130716,2713,80531,45345,0,128589,2127,79558


In [159]:
g7_jeonbuk_22.to_csv("temp2_2_governor_jeonbuk_7.csv", index=False, encoding="utf-8-sig")

## Jeonnam

In [160]:
g7_jeonnam.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김영록,득표수_3_바른미래당_박매호,득표수_4_민주평화당_민영삼,득표수_5_정의당_노형태,득표수_6_민중당_이성수,득표수_계,무효투표수,기권수
0,전라남도,합계,1577224,1092107,807902,40287,110973,37433,51410,1048005,44102,485117
1,전라남도,목포시,186864,119950,87419,2644,18579,5724,2399,116765,3185,66914
2,전라남도,여수시,235359,153554,120941,6198,10972,4184,7401,149696,3858,81805
3,전라남도,순천시,225673,144341,113180,4942,9399,4603,8612,140736,3605,81332
4,전라남도,나주시,93513,61444,47100,2317,4753,2141,2800,59111,2333,32069


In [161]:
show_column_info(g7_jeonnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_김영록
5: 득표수_3_바른미래당_박매호
6: 득표수_4_민주평화당_민영삼
7: 득표수_5_정의당_노형태
8: 득표수_6_민중당_이성수
9: 득표수_계
10: 무효투표수
11: 기권수

총 열 개수: 12
총 행 개수: 23


### v2.1

In [162]:
g7_jeonnam_21 = rename_columns_by_index(g7_jeonnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_김영록' -> '진보정당'
인덱스 5: '득표수_3_바른미래당_박매호' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [163]:
g7_jeonnam_21 = add_sum_column(g7_jeonnam_21, 6, 8, '그외정당')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_4_민주평화당_민영삼
7: 득표수_5_정의당_노형태
8: 득표수_6_민중당_이성수

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [164]:
g7_jeonnam_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주평화당_민영삼,득표수_5_정의당_노형태,득표수_6_민중당_이성수,득표수_계,무효투표수,기권수,그외정당
0,전라남도,합계,1577224,1092107,807902,40287,110973,37433,51410,1048005,44102,485117,199816
1,전라남도,목포시,186864,119950,87419,2644,18579,5724,2399,116765,3185,66914,26702
2,전라남도,여수시,235359,153554,120941,6198,10972,4184,7401,149696,3858,81805,22557


In [165]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_jeonnam_21 = g7_jeonnam_21[selected_columns].copy()

g7_jeonnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,전라남도,합계,1577224,1092107,40287,807902,199816,1048005,44102,485117
1,전라남도,목포시,186864,119950,2644,87419,26702,116765,3185,66914
2,전라남도,여수시,235359,153554,6198,120941,22557,149696,3858,81805


In [166]:
g7_jeonnam_21.to_csv("temp2_1_governor_jeonnam_7.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [167]:
g7_jeonnam_22 = rename_columns_by_index(g7_jeonnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_김영록' -> '진보정당'
인덱스 5: '득표수_3_바른미래당_박매호' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [168]:
g7_jeonnam_22 = add_sum_column(g7_jeonnam_22, 6, 8, '그외정당')
g7_jeonnam_22 = add_zero_column(g7_jeonnam_22, '무소속')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_4_민주평화당_민영삼
7: 득표수_5_정의당_노형태
8: 득표수_6_민중당_이성수

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [169]:
g7_jeonnam_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주평화당_민영삼,득표수_5_정의당_노형태,득표수_6_민중당_이성수,득표수_계,무효투표수,기권수,그외정당,무소속
0,전라남도,합계,1577224,1092107,807902,40287,110973,37433,51410,1048005,44102,485117,199816,0
1,전라남도,목포시,186864,119950,87419,2644,18579,5724,2399,116765,3185,66914,26702,0
2,전라남도,여수시,235359,153554,120941,6198,10972,4184,7401,149696,3858,81805,22557,0


In [170]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_jeonnam_22 = g7_jeonnam_22[selected_columns].copy()

g7_jeonnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,전라남도,합계,1577224,1092107,40287,807902,199816,0,1048005,44102,485117
1,전라남도,목포시,186864,119950,2644,87419,26702,0,116765,3185,66914
2,전라남도,여수시,235359,153554,6198,120941,22557,0,149696,3858,81805


In [171]:
g7_jeonnam_22.to_csv("temp2_2_governor_jeonnam_7.csv", index=False, encoding="utf-8-sig")

## Gyeongbuk

In [172]:
g7_gyeongbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_오중기,득표수_2_자유한국당_이철우,득표수_3_바른미래당_권오을,득표수_5_정의당_박창호,득표수_계,무효투표수,기권수
0,경상북도,합계,2251538,1457539,482564,732785,143409,47250,1406008,51531,793999
1,경상북도,포항시북구,223110,139955,57440,64206,10354,4719,136719,3236,83155
2,경상북도,포항시남구,198696,119400,49535,54509,8995,3564,116603,2797,79296
3,경상북도,울릉군,9057,7479,2087,4222,580,255,7144,335,1578
4,경상북도,경주시,218273,139142,44294,73050,11842,5111,134297,4845,79131


In [173]:
show_column_info(g7_gyeongbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_오중기
5: 득표수_2_자유한국당_이철우
6: 득표수_3_바른미래당_권오을
7: 득표수_5_정의당_박창호
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 25


### v2.1

In [174]:
g7_gyeongbuk_21 = rename_columns_by_index(g7_gyeongbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_오중기' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_이철우' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [175]:
g7_gyeongbuk_21 = add_sum_column(g7_gyeongbuk_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_바른미래당_권오을
7: 득표수_5_정의당_박창호

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [176]:
g7_gyeongbuk_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_권오을,득표수_5_정의당_박창호,득표수_계,무효투표수,기권수,그외정당
0,경상북도,합계,2251538,1457539,482564,732785,143409,47250,1406008,51531,793999,190659
1,경상북도,포항시북구,223110,139955,57440,64206,10354,4719,136719,3236,83155,15073
2,경상북도,포항시남구,198696,119400,49535,54509,8995,3564,116603,2797,79296,12559


In [177]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_gyeongbuk_21 = g7_gyeongbuk_21[selected_columns].copy()

g7_gyeongbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경상북도,합계,2251538,1457539,732785,482564,190659,1406008,51531,793999
1,경상북도,포항시북구,223110,139955,64206,57440,15073,136719,3236,83155
2,경상북도,포항시남구,198696,119400,54509,49535,12559,116603,2797,79296


In [178]:
g7_gyeongbuk_21.to_csv("temp2_1_governor_gyeongbuk_7.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [179]:
g7_gyeongbuk_22 = rename_columns_by_index(g7_gyeongbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_오중기' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_이철우' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [180]:
g7_gyeongbuk_22 = add_sum_column(g7_gyeongbuk_22, 6, 7, '그외정당')
g7_gyeongbuk_22 = add_zero_column(g7_gyeongbuk_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_바른미래당_권오을
7: 득표수_5_정의당_박창호

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [181]:
g7_gyeongbuk_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_권오을,득표수_5_정의당_박창호,득표수_계,무효투표수,기권수,그외정당,무소속
0,경상북도,합계,2251538,1457539,482564,732785,143409,47250,1406008,51531,793999,190659,0
1,경상북도,포항시북구,223110,139955,57440,64206,10354,4719,136719,3236,83155,15073,0
2,경상북도,포항시남구,198696,119400,49535,54509,8995,3564,116603,2797,79296,12559,0


In [182]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_gyeongbuk_22 = g7_gyeongbuk_22[selected_columns].copy()

g7_gyeongbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경상북도,합계,2251538,1457539,732785,482564,190659,0,1406008,51531,793999
1,경상북도,포항시북구,223110,139955,64206,57440,15073,0,136719,3236,83155
2,경상북도,포항시남구,198696,119400,54509,49535,12559,0,116603,2797,79296


In [183]:
g7_gyeongbuk_22.to_csv("temp2_2_governor_gyeongbuk_7.csv", index=False, encoding="utf-8-sig")

## Gyeongnam

In [184]:
g7_gyeongnam.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김경수,득표수_2_자유한국당_김태호,득표수_3_바른미래당_김유근,득표수_계,무효투표수,기권수
0,경상남도,합계,2765485,1819391,941491,765809,75418,1782718,36673,946094
1,경상남도,창원시의창구,210327,135147,72745,54066,6264,133075,2072,75180
2,경상남도,창원시성산구,184600,125797,76104,42016,6012,124132,1665,58803
3,경상남도,창원시마산합포구,154311,99281,44305,48933,4348,97586,1695,55030
4,경상남도,창원시마산회원구,166500,109609,52918,50373,4628,107919,1690,56891


In [185]:
show_column_info(g7_gyeongnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_김경수
5: 득표수_2_자유한국당_김태호
6: 득표수_3_바른미래당_김유근
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 23


### v2.1

In [186]:
g7_gyeongnam_21 = rename_columns_by_index(g7_gyeongnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_김경수' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_김태호' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [187]:
g7_gyeongnam_21 = add_sum_column(g7_gyeongnam_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_바른미래당_김유근

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [188]:
g7_gyeongnam_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_김유근,득표수_계,무효투표수,기권수,그외정당
0,경상남도,합계,2765485,1819391,941491,765809,75418,1782718,36673,946094,75418
1,경상남도,창원시의창구,210327,135147,72745,54066,6264,133075,2072,75180,6264
2,경상남도,창원시성산구,184600,125797,76104,42016,6012,124132,1665,58803,6012


In [189]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_gyeongnam_21 = g7_gyeongnam_21[selected_columns].copy()

g7_gyeongnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경상남도,합계,2765485,1819391,765809,941491,75418,1782718,36673,946094
1,경상남도,창원시의창구,210327,135147,54066,72745,6264,133075,2072,75180
2,경상남도,창원시성산구,184600,125797,42016,76104,6012,124132,1665,58803


In [190]:
g7_gyeongnam_21.to_csv("temp2_1_governor_gyeongnam_7.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [191]:
g7_gyeongnam_22 = rename_columns_by_index(g7_gyeongnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_김경수' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_김태호' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [192]:
g7_gyeongnam_22 = add_sum_column(g7_gyeongnam_22, 6, 6, '그외정당')
g7_gyeongnam_22 = add_zero_column(g7_gyeongnam_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_바른미래당_김유근

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [193]:
g7_gyeongnam_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_김유근,득표수_계,무효투표수,기권수,그외정당,무소속
0,경상남도,합계,2765485,1819391,941491,765809,75418,1782718,36673,946094,75418,0
1,경상남도,창원시의창구,210327,135147,72745,54066,6264,133075,2072,75180,6264,0
2,경상남도,창원시성산구,184600,125797,76104,42016,6012,124132,1665,58803,6012,0


In [194]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_gyeongnam_22 = g7_gyeongnam_22[selected_columns].copy()

g7_gyeongnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경상남도,합계,2765485,1819391,765809,941491,75418,0,1782718,36673,946094
1,경상남도,창원시의창구,210327,135147,54066,72745,6264,0,133075,2072,75180
2,경상남도,창원시성산구,184600,125797,42016,76104,6012,0,124132,1665,58803


In [195]:
g7_gyeongnam_22.to_csv("temp2_2_governor_gyeongnam_7.csv", index=False, encoding="utf-8-sig")

## Jeju

In [196]:
g7_jeju.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_문대림,득표수_2_자유한국당_김방훈,득표수_3_바른미래당_장성철,득표수_6_녹색당_고은영,득표수_7_무소속_원희룡,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,532515,350943,137901,11241,5019,12188,178255,344604,6339,181572
1,제주특별자치도,제주시,383917,250176,97935,8785,3874,8860,126594,246048,4128,133741
2,제주특별자치도,서귀포시,148598,100767,39966,2456,1145,3328,51661,98556,2211,47831


In [197]:
show_column_info(g7_jeju)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_문대림
5: 득표수_2_자유한국당_김방훈
6: 득표수_3_바른미래당_장성철
7: 득표수_6_녹색당_고은영
8: 득표수_7_무소속_원희룡
9: 득표수_계
10: 무효투표수
11: 기권수

총 열 개수: 12
총 행 개수: 3


### v2.1

In [198]:
g7_jeju_21 = rename_columns_by_index(g7_jeju, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_문대림' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_김방훈' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [199]:
g7_jeju_21 = add_sum_column(g7_jeju_21, 6, 8, '그외정당')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_바른미래당_장성철
7: 득표수_6_녹색당_고은영
8: 득표수_7_무소속_원희룡

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [200]:
g7_jeju_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_장성철,득표수_6_녹색당_고은영,득표수_7_무소속_원희룡,득표수_계,무효투표수,기권수,그외정당
0,제주특별자치도,합계,532515,350943,137901,11241,5019,12188,178255,344604,6339,181572,195462
1,제주특별자치도,제주시,383917,250176,97935,8785,3874,8860,126594,246048,4128,133741,139328
2,제주특별자치도,서귀포시,148598,100767,39966,2456,1145,3328,51661,98556,2211,47831,56134


In [201]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g7_jeju_21 = g7_jeju_21[selected_columns].copy()

g7_jeju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,532515,350943,11241,137901,195462,344604,6339,181572
1,제주특별자치도,제주시,383917,250176,8785,97935,139328,246048,4128,133741
2,제주특별자치도,서귀포시,148598,100767,2456,39966,56134,98556,2211,47831


In [202]:
g7_jeju_21.to_csv("temp2_1_governor_jeju_7.csv", index=False, encoding="utf-8-sig")

### v2.2

In [203]:
g7_jeju_22 = rename_columns_by_index(g7_jeju, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_문대림' -> '진보정당'
인덱스 5: '득표수_2_자유한국당_김방훈' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [204]:
g7_jeju_22 = add_sum_column(g7_jeju_22, 6, 7, '그외정당')
g7_jeju_22 = add_sum_column(g7_jeju_22, 8, 8, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_바른미래당_장성철
7: 득표수_6_녹색당_고은영

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13

=== 합산할 열들 (인덱스 8~8) ===
8: 득표수_7_무소속_원희룡

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 14


In [205]:
g7_jeju_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_바른미래당_장성철,득표수_6_녹색당_고은영,득표수_7_무소속_원희룡,득표수_계,무효투표수,기권수,그외정당,무소속
0,제주특별자치도,합계,532515,350943,137901,11241,5019,12188,178255,344604,6339,181572,17207,178255
1,제주특별자치도,제주시,383917,250176,97935,8785,3874,8860,126594,246048,4128,133741,12734,126594
2,제주특별자치도,서귀포시,148598,100767,39966,2456,1145,3328,51661,98556,2211,47831,4473,51661


In [206]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g7_jeju_22 = g7_jeju_22[selected_columns].copy()

g7_jeju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,532515,350943,11241,137901,17207,178255,344604,6339,181572
1,제주특별자치도,제주시,383917,250176,8785,97935,12734,126594,246048,4128,133741
2,제주특별자치도,서귀포시,148598,100767,2456,39966,4473,51661,98556,2211,47831


In [207]:
g7_jeju_22.to_csv("temp2_2_governor_jeju_7.csv", index=False, encoding="utf-8-sig")

# Batch CSV Files to ZIP

In [208]:
import zipfile
import glob

# Find all CSV files in current directory
csv_files = glob.glob('*.csv')

# Create ZIP file
with zipfile.ZipFile('all_csv_files.zip', 'w') as zipf:
   for file in csv_files:
       zipf.write(file)
       print(f"Added: {file}")  # Show progress

print(f"Total {len(csv_files)} files compressed.")

Added: temp2_2_governor_incheon_7.csv
Added: temp2_1_governor_ulsan_7.csv
Added: temp2_2_governor_busan_7.csv
Added: temp2_2_governor_daejeon_7.csv
Added: temp2_2_governor_gangwon_7.csv
Added: temp2_2_governor_sejong_7.csv
Added: temp2_1_governor_gangwon_7.csv
Added: temp2_1_governor_gyeonggi_7.csv
Added: temp2_1_governor_chungbuk_7.csv
Added: temp2_1_governor_daegu_7.csv
Added: temp2_1_governor_jeonbuk_7.csv
Added: temp2_2_governor_chungbuk_7.csv
Added: temp2_2_governor_gyeonggi_7.csv
Added: temp2_1_governor_sejong_7.csv
Added: temp2_2_governor_jeonnam_7.csv
Added: temp2_1_governor_gyeongbuk_7.csv
Added: temp2_1_governor_chungnam_7.csv
Added: temp2_1_governor_incheon_7.csv
Added: temp2_1_governor_daejeon_7.csv
Added: temp2_1_governor_busan_7.csv
Added: temp2_1_governor_seoul_7.csv
Added: temp2_2_governor_gyeongnam_7.csv
Added: temp2_2_governor_jeonbuk_7.csv
Added: temp2_2_governor_seoul_7.csv
Added: temp2_2_governor_chungnam_7.csv
Added: temp2_2_governor_ulsan_7.csv
Added: temp2_1_gov